In [15]:
import pandas as pd

df=pd.read_csv("feasibility_bianeu_clean.csv")

#drop 'base_model' human
df = df[df['base_model'] != 'human']

#drop 'scenario' BIASED
df = df[~df['scenario'].isin(['BIAS_WORD', 'BIAS_EXAMPLE', 'BIAS_NUM_HIGH', 'BIAS_NUM_LOW'])]

#CHANGE 'SCENARIO' VALUE FROM NEUTRAL TO CONTEXT
df['scenario'] = df['scenario'].replace('NEU', 'CONTEXT')

#DROP COLUMNS
df = df.drop(columns=['bias_type', 'anchor_level', 'id_key'])

#save new dataframe
df_new = df.copy()
df_new.to_csv("feasibility_context_llm.csv", index=False)
print(df.shape)
print(df_new.shape)

(1200, 10)
(1200, 10)


In [10]:
import pandas as pd
df=pd.read_csv("merged_feasibility_zeroshot.csv")

#drop where 'source' == human
df = df[df['source'] != 'human']

#save csv
df.to_csv("merged_feasibility_zeroshot_llm.csv", index=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 22 to 1221
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   row_id      1200 non-null   object 
 1   source      1200 non-null   object 
 2   condition   1200 non-null   object 
 3   base_model  1200 non-null   object 
 4   variant_id  1200 non-null   object 
 5   model       1200 non-null   object 
 6   rating      1200 non-null   int64  
 7   label       1200 non-null   object 
 8   iteration   1200 non-null   float64
 9   timestamp   1200 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 103.1+ KB


In [17]:
df2=pd.read_csv("context_feasibility_all.csv")
df=pd.read_csv("merged_feasibility_zeroshot_llm.csv")

#compare columns
df2.columns.difference(df.columns)
print(df2.columns.difference(df.columns))

#drop raw_response from df2
df2 = df2.drop(columns=['raw_response'], errors='ignore')

#save
df2.to_csv("context_feasibility_50iter_cleaned.csv", index=False)

df.info()

Index(['raw_response'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   row_id      1200 non-null   object 
 1   source      1200 non-null   object 
 2   condition   1200 non-null   object 
 3   base_model  1200 non-null   object 
 4   variant_id  1200 non-null   object 
 5   model       1200 non-null   object 
 6   rating      1200 non-null   int64  
 7   label       1200 non-null   object 
 8   iteration   1200 non-null   float64
 9   timestamp   1200 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 93.9+ KB


In [13]:
import pandas as pd

df = pd.read_csv("context_feasibility_50iter_cleaned.csv")

# ensure iteration is numeric
df["iteration"] = pd.to_numeric(df["iteration"], errors="coerce")

# filter 0..49 (inclusive)
df_0_49 = df[(df["iteration"] >= 0) & (df["iteration"] <= 49)]

# save
df_0_49.to_csv("context_feasibility_50iter_cleaned_iter0_49.csv", index=False)

print(f"Kept {len(df_0_49)} rows out of {len(df)} → context_feasibility_50iter_cleaned_iter0_49.csv")

df_0_49.info()


Kept 1200 rows out of 1254 → context_feasibility_50iter_cleaned_iter0_49.csv
<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 1217
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   row_id      1200 non-null   object
 1   variant_id  1200 non-null   object
 2   base_model  1200 non-null   object
 3   model       1200 non-null   object
 4   rating      1200 non-null   int64 
 5   label       1200 non-null   object
 6   iteration   1200 non-null   int64 
 7   timestamp   1200 non-null   object
dtypes: int64(2), object(6)
memory usage: 84.4+ KB


In [14]:
import pandas as pd

df=pd.read_csv("merged_feasibility_zeroshot_llm.csv")
df2=pd.read_csv("context_feasibility_50iter_cleaned_iter0_49.csv")

#compare columns
df2.columns.difference(df.columns)
print(df2.columns.difference(df.columns))

Index([], dtype='object')


In [44]:

import pandas as pd

df1 = pd.read_csv("context_feasibility_50iter_cleaned_iter0_49.csv")
df2 = pd.read_csv("feasibility_zeroshhot_complete.csv")

# 1) Align df1 to df2's schema
df1_aligned = df1.copy()

# add constant columns required by df2
df1_aligned["source"] = "llm"
df1_aligned["condition"] = "CONTEXT"

# 2) Reorder columns to exactly match df2
target_cols = [
    "row_id", "source", "condition", "base_model",
    "variant_id", "model", "rating", "label", "iteration", "timestamp"
]

# ensure all target columns exist (create empty if missing)
for c in target_cols:
    if c not in df1_aligned.columns:
        df1_aligned[c] = pd.NA

# select & order
df1_aligned = df1_aligned[target_cols]

# 3) Also coerce df2 to the same column order (just in case)
df2_aligned = df2[target_cols]

# 4) Concatenate (row-wise)
combined = pd.concat([df2_aligned, df1_aligned], ignore_index=True)

# save
combined.to_csv("merged_feasibility_zeroshot_context.csv", index=False)
print(f"Combined rows: {len(combined)}  → saved to merged_feasibility_zeroshot_context.csv")

#combined.info()

Combined rows: 2400  → saved to merged_feasibility_zeroshot_context.csv


In [34]:
import pandas as pd
df=pd.read_csv("merged_feasibility_zeroshot_llm.csv")
# df = pd.read_csv("your_file.csv")
pattern = r'^(?P<head>[^_\n]*?)-[^_\n]+(?=_)'
repl    = r'\g<head>'

df["variant_id"] = df["variant_id"].astype(str).str.replace(pattern, repl, regex=True)
df["row_id"]     = df["row_id"].astype(str).str.replace(pattern, repl, regex=True)

df.to_csv("mrged_feasibility_zeroshhot.csv", index=False)
#this new_merged...csv, came into use due to challenge of naming that happened in two base models: gemma3, and llama
#therefore this is the correct version to use for the analysis

In [26]:
#select distinct values in column base_model
dis=pd.read_csv("new_merged_feasibility_zero_context.csv")
dis = dis["base_model"].unique()

print(dis)

['llama-pro' 'phi4' 'mistral' 'gemma3-12b' 'llama_generalist'
 'llama_generalist2' 'llama_normative' 'llama_normative2'
 'llama_subject_matter' 'llama_subject_matter2' 'mistral_generalist'
 'mistral_generalist2' 'mistral_normative' 'mistral_normative2'
 'mistral_subject_matter' 'mistral_subject_matter2' 'gemma3_generalist'
 'gemma3_generalist2' 'gemma3_normative' 'gemma3_normative2'
 'gemma3_subject_matter' 'gemma3_subject_matter2' 'llama' 'gemma3']


In [30]:

df1 = pd.read_csv("context_feasibility_50iter_cleaned.csv")


df1.info()

#check info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   row_id      1200 non-null   object
 1   variant_id  1200 non-null   object
 2   base_model  1200 non-null   object
 3   model       1200 non-null   object
 4   rating      1200 non-null   int64 
 5   label       1200 non-null   object
 6   iteration   1200 non-null   int64 
 7   timestamp   1200 non-null   object
dtypes: int64(2), object(6)
memory usage: 75.1+ KB


In [41]:
df2 = pd.read_csv("mrged_feasibility_zeroshhot.csv")

# change values in base_model column only
df2["base_model"] = df2["base_model"].str.replace("gemma3-12b", "gemma3")
df2["base_model"] = df2["base_model"].str.replace("llama-pro", "llama")

# save to csv
df2.to_csv("feasibility_zeroshhot_complete.csv", index=False)

# distinct base models
df2["base_model"].unique()

print(df2["base_model"].unique())


['llama' 'phi4' 'mistral' 'gemma3']


In [3]:
import pandas as pd
df=pd.read_csv("new_merged_feasibility_zero_context.csv")

#inspect missing values in 'rating' column
df['condition'].isnull().sum()

print(df['condition'].isnull().sum() > 0)

False
